# Data Gathering

This notebook gathers the meta-data for the evaluation. It runs the various respiratory extraction methods.

In [ ]:
import os

import numpy as np

import respiratory_extraction.dataset as repository

dataset = repository.from_default()
subjects = dataset.get_subjects()
subjects

In [ ]:
# The scenarios in which the subjects were recorded
# scenarios = dataset.get_scenarios()
# scenarios = [
#     '101_natural_lighting',
#     # '102_artificial_lighting',
#     '103_abrupt_changing_lighting',
#     '104_dim_lighting_auto_exposure',
#     # '106_green_lighting',
#     # '107_infrared_lighting',
#     '201_shouldercheck',
#     '202_scale_movement',
#     '203_translation_movement',
#     # '204_writing'
# ]
scenarios = [
    '101_natural_lighting',
]
scenarios

In [ ]:
from datetime import datetime

hyperparameters = {
    'quality_level': 0.1,
    'quality_level_rv': 0.05,
}

evaluation_metadata = {
    'start_time': datetime.now(),
    'subjects': subjects,
    'scenarios': scenarios,
}

In [ ]:
import respiratory_extraction.utils.roi as roi

yolo = roi.YOLO()


def get_rois(frame: np.ndarray) -> list[tuple[np.ndarray, str]]:
    """
    Get the regions of interest (ROIs) for the given frame
    :param frame: The frame to get the ROIs from
    :return: A list of tuples containing the ROI and the name of the ROI
    """

    regions = [
        # ROI for the full frame
        ((0, 0, frame.shape[1], frame.shape[0]), 'full')
    ]

    # Calculate the region of interest (ROI) based on the face
    faces = roi.detect_faces(frame)
    if len(faces) == 1:
        chest_roi = roi.roi_from_face(faces[0])
        regions.append((chest_roi, 'chest'))

    # Use the detected person to create a mask
    persons = yolo.detect_classes(frame, clazz='person')
    if len(persons) == 1:
        regions.append((persons[0], 'person'))

    return regions

In [ ]:
from respiratory_extraction.models.signal_extraction import pixel_intensity, optical_flow

extracted_signals = []
ground_truth_signals = []

for subject in subjects:
    for idx, scenario in enumerate(scenarios):
        print(f'Processing {subject} - {scenario}')

        frames, params = dataset.read_video_gray(subject, scenario)

        rois = get_rois(frames[0])

        gt_signal, gt_sampling_rate = dataset.get_ground_truth_rr_signal(subject, scenario)
        ground_truth_signals.append({
            'subject': subject,
            'scenario': scenario,
            'signal': gt_signal.tolist(),
            'sampling_rate': gt_sampling_rate,
        })

        for region in rois:
            roi_area, roi_name = region

            #
            # Calculate the average pixel intensity
            #

            pi_start = datetime.now()
            pi_signal = pixel_intensity.average_pixel_intensity(frames, roi=roi_area)
            extracted_signals.append({
                'subject': subject,
                'scenario': scenario,
                'method': 'pixel_intensity',
                'parameters': {
                    'roi': roi_name,
                    'roi_area': list(roi_area),
                },
                'execution_time': datetime.now() - pi_start,
                'sampling_rate': params.fps,
                'signal': pi_signal.tolist(),
            })

            #
            # Calculate the optical flow without cgof
            # 

            of_raw_start = datetime.now()
            of_signal_raw = optical_flow.extract_signal(
                frames,
                roi=roi_area,
                quality_level=hyperparameters['quality_level'],
                quality_level_rv=hyperparameters['quality_level_rv'],
                use_cgof=False,
            )
            extracted_signals.append({
                'subject': subject,
                'scenario': scenario,
                'method': 'optical_flow',
                'parameters': {
                    'roi': roi_name,
                    'roi_area': list(roi_area),
                    'quality_level': hyperparameters['quality_level'],
                    'quality_level_rv': hyperparameters['quality_level_rv'],
                    'use_cgof': False
                },
                'execution_time': datetime.now() - of_raw_start,
                'sampling_rate': params.fps,
                'signal': of_signal_raw.tolist(),
            })

            #
            # Calculate the optical flow with cgof
            #

            of_cgof_start = datetime.now()
            of_signal_raw = optical_flow.extract_signal(
                frames,
                roi=roi_area,
                quality_level=hyperparameters['quality_level'],
                quality_level_rv=hyperparameters['quality_level_rv'],
                use_cgof=True,
            )

            extracted_signals.append({
                'subject': subject,
                'scenario': scenario,
                'method': 'optical_flow',
                'parameters': {
                    'roi': roi_name,
                    'roi_area': list(roi_area),
                    'quality_level': hyperparameters['quality_level'],
                    'quality_level_rv': hyperparameters['quality_level_rv'],
                    'use_cgof': True
                },
                'execution_time': datetime.now() - of_cgof_start,
                'sampling_rate': params.fps,
                'signal': of_signal_raw.tolist(),
            })

        # Garbage collect the frames
        del frames

In [ ]:
evaluation_metadata['end_time'] = datetime.now()

In [ ]:
import pandas as pd

df = pd.DataFrame(extracted_signals)
df.head()

In [ ]:
import respiratory_extraction.utils as utils

evaluation_dir = os.path.join(os.getcwd(), '..', 'evaluation', 'signals')
if not os.path.exists(evaluation_dir):
    os.makedirs(evaluation_dir)

# Save the extracted_signals as a JSON
json_path = os.path.join(evaluation_dir, 'predictions.json')
utils.write_json(json_path, extracted_signals)

# Save the evaluation dataframe
csv_path = os.path.join(evaluation_dir, 'predictions.csv')
df.to_csv(csv_path, index=False)

# Save the hyperparameters as prettified json
json_path = os.path.join(evaluation_dir, 'parameters.json')
utils.write_json(json_path, evaluation_metadata)

# Save the ground truth signals as a JSON
json_path = os.path.join(evaluation_dir, 'ground_truth.json')
utils.write_json(json_path, ground_truth_signals)

# Save the ground truth signals as a CSV
df_gt = pd.DataFrame(ground_truth_signals)
csv_path = os.path.join(evaluation_dir, 'ground_truth.csv')
df_gt.to_csv(csv_path, index=False)